# RNN Signal - With Patient

Author: Jake Dumbauld <br>
Contact: jacobmilodumbauld@gmail.com<br>
Date: 3.15.22

In [1]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import re

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, GRU
from tensorflow.keras import regularizers
import keras_tuner as kt

In [2]:
import random as python_random

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers in a well-defined state.
python_random.seed(42)

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(42)

#not sure if the below are necessary - leaving in to perhaps un-comment later.
%env PYTHONHASHSEED=0
%env CUDA_VISIBLE_DEVICES=""

env: PYTHONHASHSEED=0
env: CUDA_VISIBLE_DEVICES=""


## Helper Functions

### Model Loss

In [3]:
def graph_model_loss(title, history):
    """
    Description:
    Graphs training vs validation loss over epochs for a given model. 
    
    History: tensorflow.python.keras.callbacks.History object
    Title: str
    """ 
    plt.figure(figsize=(12,8))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(title,size=24)
    plt.ylabel('Loss',size=16)
    plt.xlabel('Epoch',size=16)
    plt.legend(['Train', 'Validation'])
    plt.show()

### Train/Test Acc Printout

In [4]:
def evaluate_model(model, history):
    """
    Description:
    Outputs model train & test accuracies for currently defined train and test set variables.
    
    model: tensorflow model,
    history: tensorflow.python.keras.callbacks.History object
    """
    # Evaluate the network
    train_accuracy = history.history["binary_accuracy"][-1]
    result = model.evaluate(X_test,y_test, verbose=1)

    print(f"Train Accuracy: {np.round(train_accuracy, 6)*100}%")
    print(f"Test Accuracy: {np.round(result[1], 6)*100}%")

### Defining Search Space

In [5]:
def build_RNN_model(hp):
    model = keras.Sequential()
    
    model.add(
        LSTM(units=hp.Int('LSTM_units1', min_value=16, max_value=32, step=8),
        activation='relu',
        return_sequences=True
        )
    )
    #Tuning whether or not to use dropout.
    if hp.Boolean("LSTM_dropout1"):
        model.add(Dropout(rate=0.25))
                  
    model.add(
        LSTM(units=hp.Int('LSTM_units2', min_value=8, max_value=16, step=4),
        activation='relu',
        return_sequences=False
        )
    )
    #Tuning whether or not to use dropout.
    if hp.Boolean("LSTM_dropout2"):
        model.add(Dropout(rate=0.25))
            
    for i in range(hp.Int('dense_layers', 1, 3)):
        model.add(
            Dense(
            #Tuning the number of units in my input layer.
            units=hp.Int("units" + str(i), min_value=16, max_value=64, step=8),
            activation="relu"
            )
        )
        #Tuning whether or not to use dropout.
        if hp.Boolean("dropout" + str(i)):
            model.add(Dropout(rate=0.25))

        #Turning whether or not to add batch normalization
        if hp.Boolean("normalization" + str(i)):
            model.add(BatchNormalization())

    #output layer
    model.add(Dense(1, activation="sigmoid"))
    
    #defining learning rate
    lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
                      #tuning initial learning rate
                      initial_learning_rate=hp.Float("starting_learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"),
                      decay_steps=1.0,
                      decay_rate=0.1
                  )
    model.compile(
        #Optimizer
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule),
        #Loss
        loss=keras.losses.BinaryCrossentropy(),
        #Metrics
        metrics=[keras.metrics.BinaryAccuracy()]
    )
    return model

build_RNN_model(kt.HyperParameters())

2022-03-31 18:16:52.953447: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
X = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/signal_withPatient.npy', allow_pickle=True)
y = np.load('/Users/jmd/Documents/BOOTCAMP/Capstone/arrays/target_array.npy', allow_pickle=True)

In [7]:
X =X.reshape(X.shape[0],X.shape[2],X.shape[1])

In [11]:
X.shape

(3007, 6144, 11)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.3)

In [9]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [10]:
tuner = kt.BayesianOptimization(
    hypermodel=build_RNN_model,
    objective="val_loss",
    max_trials=25,
    seed=42,
    overwrite=True,
    directory='/Users/jmd/Documents/BOOTCAMP/Capstone/kerastune_searches',
    project_name='RNN_signal_no_patient'
)

tuner.search(X_train, y_train, epochs=50, validation_data=(X_val,y_val), callbacks=[es_callback])


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
LSTM_units1       |24                |?                 
LSTM_dropout1     |True              |?                 
LSTM_units2       |12                |?                 
LSTM_dropout2     |False             |?                 
dense_layers      |3                 |?                 
units0            |32                |?                 
dropout0          |False             |?                 
normalization0    |True              |?                 
starting_learni...|0.00098872        |?                 



2022-03-31 18:16:54.765373: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/50
17/46 [==========>...................] - ETA: 1:25 - loss: nan - binary_accuracy: 0.7987

KeyboardInterrupt: 

In [ ]:
tuner.results_summary(num_trials=1)

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# Get the best hyperparameters.
best_hps = tuner.get_best_hyperparameters()
# Build the model with the best hp.
best_model = build_model(best_hps[0])

history = best_model.fit(X_train, y_train, epochs=100, validation_data=(X_val,y_val), callbacks=[es_callback])

In [ ]:
best_model.summary()

In [ ]:
evaluate_model(best_model, history)

In [ ]:
graph_model_loss('LSTM Signal w/o Patient Information', sequential_MFCC_with_patient_history)

In [ ]:
#saving model
sequential_MFCC_with_patient.save('/Users/jmd/Documents/BOOTCAMP/Capstone/neural_nets/LSTM_signal_no_patient', overwrite=True)